In [18]:
import json, yaml
import os
from datetime import datetime
from utils.gen_link_info import gen_port_info

In [19]:
# set env vars
os.environ['MLIB_DEVEL_PATH'] = '/home/wei/scilab_demo/mlib_devel'
os.environ['HDL_ROOT'] = '/home/wei/scilab_demo/mlib_devel/scilab_library/hdl_sources'

In [42]:
# write the blkinfo to the dict
def write_blkinfo_to_dict(blk_info, blk_tmp):
    # make a copy of the dict
    template = blk_tmp.copy()
    keys = list(template.keys())
    index = blk_info['id']
    for i in range(len(index)):
        try:
            template[keys[index[i][0]]] = int(blk_info['val'][i][0])
        except:
            try:
                template[keys[index[i][0]]] = float(blk_info['val'][i][0])
            except:
                if blk_info['val'][i][0] == 'on':
                    template[keys[index[i][0]]] = True
                elif blk_info['val'][i][0] == 'off':
                    template[keys[index[i][0]]] = False
                else:
                    template[keys[index[i][0]]] = blk_info['val'][i][0]
    # go through all of the k-v pairs in the template, 
    # and modify all of the 'on' to True and 'off' to False
    for k, v in template.items():
        if v == 'on':
            template[k] = True
        elif v == 'off':
            template[k] = False
    return template

def gen_verilog_module(module_name, in_port, out_port, bitwidth = 1, path = './'):
    '''
    Description: 
        Generate user module for jasper.
        The module only has one in_port, but it may have multiple out_ports.
        The only one in_port drives all of the out_ports.
    Inputs:
        - module_name(str): the name of the user module
        - in_ports(list): the names of the in_ports
        - out_ports(list): the names of the out_ports
        - bitwidth(int): the bitwidth of the in_port
        - path(str): the path to save the user module
    '''
    src_str = []
    # add module name
    src_str.append('module %s('%module_name)
    # add clock port
    src_str.append('input clk,')
    # add in_ports
    src_str.append('input [%d:0] %s,'%(bitwidth-1, in_port))
    # add out_ports
    src_str.append('output [%d:0] %s,'%(bitwidth-1, out_port))
    # remove the last comma
    src_str[-1] = src_str[-1][:-1]
    # add endmodule
    src_str.append(');\n')
    # add logic
    src_str.append('assign %s = %s;'%(out_port, in_port))
    # add endmodule
    src_str.append('')
    src_str.append('endmodule')
    # write to file
    with open('%s/%s.v'%(path, module_name), 'w') as f:
        f.write('\n'.join(src_str))

def gen_glue_module(link_info,file_dir='./'):
    module_name = link_info['link_type'] + '_' + link_info['src_blk_name'] + '_' + link_info['dst_blk_name']
    in_port = link_info['src_port_name']
    out_port = link_info['dst_port_name']
    if link_info['src_port_width'] != link_info['dst_port_width']:
        print('port width not match')
        return
    bitwidth = link_info['src_port_width']
    gen_verilog_module(module_name, in_port, out_port, bitwidth, file_dir)

In [36]:
# get the scilab path from the environment variable
mlib_devel_path = os.getenv('MLIB_DEVEL_PATH')
scilab_library_path = mlib_devel_path + '/scilab_library'
# open the jasper.json file
with open('%s/jasper.json'%mlib_devel_path) as f:
    blkinfo = json.load(f)
# after reading the file, remove it
# os.remove('jasper.json')
# open the block_info_template.json file
with open('%s/block_info_template.json'%(scilab_library_path)) as f:
    blkinfo_template = json.load(f)

In [37]:
blkfn = blkinfo['project']['filename']
proj_name = blkfn.split('/')[-1].split('.')[0]
filepath = blkfn.split('.')[0]

In [38]:
# get the scilab blk, blk objs and link objs from the jaser.json file
blk_objs = []
for k, v in blkinfo.items():
    if k.startswith('blk'):
        blk_objs.append(v)
link_info = blkinfo['link_info']
# genenrate yellow blocks dict
yellow_blocks = {}
for blk in blk_objs:
    tag = blk['tag']
    # check if the block in the blockinfo starts with xps
    if blkinfo_template[tag]['tag'].startswith('xps'):
        # populate yellow block
        key = proj_name + '/' + blk['val'][0][0]
        yellow_blocks[key] = write_blkinfo_to_dict(blk, blkinfo_template[tag])
        yellow_blocks[key]['fullpath'] = key
user_modules = {}
jasper_per = {}
jasper_per['yellow_blocks'] = yellow_blocks
jasper_per['user_modules'] = user_modules

In [39]:
# now, we need to generate jasper.dsp, which contains the dsp block info
dsp_blocks = {}
for blk in blk_objs:
    tag = blk['tag']
    # check if the block in the blockinfo starts with dsp or xps:xsg
    # we need xps:xsg as it contains the platform info
    if blkinfo_template[tag]['tag'].startswith('dsp') or blkinfo_template[tag]['tag'] == 'xps:xsg':
        # populate dsp block
        key = proj_name + '/' + blk['val'][0][0]
        dsp_blocks[key] = write_blkinfo_to_dict(blk, blkinfo_template[tag])
        dsp_blocks[key]['fullpath'] = key
# generate jasper.dsp
jasper_dsp = {}
jasper_dsp['dsp_blocks'] = dsp_blocks

In [40]:
file_dir = '/home/wei/Projects/scilab_test/adder_test'

In [45]:
for link in link_info:
    if link['link_type'] == 'xps_xps' or link['link_type'] == 'dsp_dsp':
        gen_glue_module(link, file_dir)

In [27]:
link_info[0]

{'src_blk_tag': 'swreg',
 'src_blk_name': 'swreg0',
 'src_blk_id': 1,
 'src_port_name': 'adder_test_swreg0_user_data_out',
 'src_port_id': 1,
 'src_port_width': 32,
 'dst_blk_tag': 'adder',
 'dst_blk_name': 'adder',
 'dst_blk_id': 5,
 'dst_port_name': 'adder_test_adder_in1',
 'dst_port_id': 1,
 'dst_port_width': 32,
 'link_type': 'xps_dsp'}